In [1]:
import re
import os
import random

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
path = 'data'
data = ''

# opening each file and appending to data
# files = os.listdir(path)
# for file in files:
#     if os.path.isfile(os.path.join(path, file)):
#         file_content = open(os.path.join(path, file), 'r', encoding='utf-8').read()
#         data += file_content

files = ['bieber', 'bruno-mars', 'drake', 'rihanna', 'adele']
for file in files:
    if os.path.isfile(os.path.join(path, file + '.txt')):
        file_content = open(os.path.join(path, file + '.txt'), 'r', encoding='utf-8').read()
        data += file_content

In [3]:
print('Length of corpus:', len(data))
print(data[:200])

Length of corpus: 711225
What do you mean?
Oh, oh, oh
When you sometimes say yes
But you sometimes say no
What do you mean?
Hey, yeah
When you don't want me to move
But you tell me to go
What do you mean?
Oh
What do you mean?


In [4]:
print('Number of unique characters before:', len(set(data)))

# Replace all non ascii characters in data with ''
data = re.sub(r'[^\x00-\x7F]', r'', data)
print('Number of unique characters after:', len(set(data)))

Number of unique characters before: 90
Number of unique characters after: 82


In [5]:
chars = sorted(set(data))
print('Total chars:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Total chars: 82


In [6]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(data) - maxlen, step):
    sentences.append(data[i: i + maxlen])
    next_chars.append(data[i + maxlen])

print('Number of sequences:', len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)
for i, sent in enumerate(sentences):
    for t, char in enumerate(sent):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 236991


In [7]:
model = Sequential([
    InputLayer(input_shape = (maxlen, len(chars))),
    LSTM(units = 128, activation = 'tanh', return_sequences = True),
    Dropout(0.2),
    LSTM(units = 128, activation = 'tanh'),
    Dense(units = len(chars), activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 128)           108032    
_________________________________________________________________
dropout (Dropout)            (None, 40, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 82)                10578     
Total params: 250,194
Trainable params: 250,194
Non-trainable params: 0
_________________________________________________________________


In [8]:
optimizer = RMSprop(learning_rate = 1e-3, decay = 1e-5)
model.compile(loss = "categorical_crossentropy", optimizer = optimizer)

In [9]:
len(tf.config.list_physical_devices('GPU'))

1

In [10]:
epochs = 200
batch_size = 128

callback = ModelCheckpoint(filepath = 'checkpoints/weights.{epoch:02d}-{loss:.2f}.hdf5', monitor = 'loss', save_best_only = True, mode = 'min')
model.fit(x, y, batch_size = batch_size, epochs = epochs, callbacks = [callback])

Epoch 1/200
1852/1852 [==============================] - 35s 15ms/step - loss: 2.4415
Epoch 2/200
1852/1852 [==============================] - 30s 16ms/step - loss: 1.9205
Epoch 3/200
1852/1852 [==============================] - 28s 15ms/step - loss: 1.7345
Epoch 4/200
1852/1852 [==============================] - 29s 15ms/step - loss: 1.6245
Epoch 5/200
1852/1852 [==============================] - 28s 15ms/step - loss: 1.5518
Epoch 6/200
1852/1852 [==============================] - 28s 15ms/step - loss: 1.4930
Epoch 7/200
1852/1852 [==============================] - 31s 17ms/step - loss: 1.4478
Epoch 8/200
1852/1852 [==============================] - 29s 16ms/step - loss: 1.4106
Epoch 9/200
1852/1852 [==============================] - 28s 15ms/step - loss: 1.3769
Epoch 10/200
1852/1852 [==============================] - 29s 16ms/step - loss: 1.3473
Epoch 11/200
1852/1852 [==============================] - 29s 16ms/step - loss: 1.3220
Epoch 12/200
1852/1852 [============================

In [11]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
start_index = random.randint(0, len(data) - maxlen - 1)

generated = ""
sentence = data[start_index : start_index + maxlen]
print('Generating with seed: "' + sentence + '"')

for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
        
    preds = model.predict(x_pred)[0]

    next_index = sample(preds)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated += next_char

print(generated)

Generating with seed: "ver sinkin'
'Bout to set it off, in this"


C:\Users\Sanket\AppData\Local\Temp/ipykernel_13016/1116348211.py:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds)


 charch
This feeling up
All the girl along
See up to purply lights On the mistow
Why with you to sind me
I wanna make o long off
Nobody that happen me to see What do you mean
more you smece
I feel you thinks I know How I would rew a Lour bust into right
You got a feelings and your heart
Whore she's the only one (girl better wantine)
Sayes you love me at the frink with the damb
Shownlies where you 


In [14]:
model.save('model_200')

INFO:tensorflow:Assets written to: model_200\assets


INFO:tensorflow:Assets written to: model_200\assets
